# RAG Model 

The implementation fo the RAG model for the assignment.

In [5]:
# from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration
# import torch
# from datasets import load_from_disk
# device = torch.device("cuda:0")

# ds = load_from_disk("/workspace/data/filtered_small_embeddings")["train"]
# def get_title_examples(examples):
#     split_title = [ i.split("\n")[0] for i in examples["sentence_split"]]
#     examples["title"] = split_title
#     return examples
# ds = ds.map(get_title_examples, batched=True, num_proc=32)
# ds

In [13]:
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration
import torch
from datasets import load_from_disk
device = torch.device("cuda:0")

ds = load_from_disk("/workspace/data/filtered_split_sentences_embeddings")
ds

Dataset({
    features: ['date', 'hash', 'title', 'sentence', 'embeddings'],
    num_rows: 3034846
})

In [16]:
import datasets
ds = datasets.DatasetDict({"train":ds})

In [17]:
len(ds["train"][0]["embeddings"])

384

In [9]:
# ds[0]["embeddings"].shape

In [10]:
# def get_title_examples(examples):
#     split_title = [ i.split("\n")[0] for i in examples["sentence_split"]]
#     examples["title"] = split_title
#     return examples
# ds = ds.map(get_title_examples, batched=True, num_proc=32)
# ds

In [19]:
ds["train"] = ds["train"].add_faiss_index(column='embeddings')

100%|██████████| 3035/3035 [00:12<00:00, 246.35it/s]


In [20]:
ds

DatasetDict({
    train: Dataset({
        features: ['date', 'hash', 'title', 'sentence', 'embeddings'],
        num_rows: 3034846
    })
})

In [21]:
ds = ds.rename_column("sentence","text")

In [22]:
ds

DatasetDict({
    train: Dataset({
        features: ['date', 'hash', 'title', 'text', 'embeddings'],
        num_rows: 3034846
    })
})

In [24]:
# ds[1000]

In [25]:
faiss_index = ds.get_index('embeddings').faiss_index

AttributeError: 'DatasetDict' object has no attribute 'get_index'

In [26]:
# from transformers import RagConfig
# config = RagConfig.from_pretrained(
#     "facebook/rag-sequence-nq")

In [27]:
# dict(config)

In [29]:
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration
import torch

tokenizer = RagTokenizer.from_pretrained("facebook/rag-sequence-nq")
retriever = RagRetriever.from_pretrained(
    "facebook/rag-sequence-nq", indexed_dataset=ds["train"],retrieval_vector_size=364
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

In [30]:
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration
model = RagSequenceForGeneration.from_pretrained("facebook/rag-token-nq", retriever=retriever,retrieval_vector_size=364)

/opt/conda/lib/python3.8/site-packages/transformers/models/bart/configuration_bart.py:179: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(
Some weights of the model checkpoint at facebook/rag-token-nq were not used when initializing RagSequenceForGeneration: ['rag.question_encoder.question_encoder.bert_model.pooler.dense.bias', 'rag.question_encoder.question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing RagSequenceForGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RagSequenceForGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassificati

In [31]:
model.config

RagConfig {
  "_commit_hash": "af32fa164f774a532dfb63c94b2e898e80434643",
  "_name_or_path": "facebook/rag-token-nq",
  "architectures": [
    "RagTokenForGeneration"
  ],
  "bad_words_ids": [
    [
      0,
      0
    ]
  ],
  "bos_token_id": 0,
  "dataset": "wiki_dpr",
  "dataset_split": "train",
  "decoder_start_token_id": 2,
  "do_deduplication": true,
  "do_marginalize": false,
  "doc_sep": " // ",
  "eos_token_id": 2,
  "exclude_bos_score": false,
  "forced_eos_token_id": 2,
  "generator": {
    "_name_or_path": "",
    "_num_labels": 3,
    "activation_dropout": 0.0,
    "activation_function": "gelu",
    "add_bias_logits": false,
    "add_cross_attention": false,
    "add_final_layer_norm": false,
    "architectures": [
      "BartForConditionalGeneration"
    ],
    "attention_dropout": 0.0,
    "bad_words_ids": null,
    "begin_suppress_tokens": null,
    "bos_token_id": 0,
    "chunk_size_feed_forward": 0,
    "classif_dropout": 0.0,
    "classifier_dropout": 0.0,
    "cros

In [32]:
inputs = tokenizer('Who was the Labour candidate in Norwich South?', return_tensors="pt")
input_ids = inputs["input_ids"]
question_hidden_states = model.question_encoder(input_ids)[0]
# 2. Retrieve
docs_dict = retriever(input_ids.numpy(), question_hidden_states.detach().numpy(), return_tensors="pt")
doc_scores = torch.bmm(
    question_hidden_states.unsqueeze(1), docs_dict["retrieved_doc_embeds"].float().transpose(1, 2)
).squeeze(1)
# 3. Forward to generator
outputs = model(
    context_input_ids=docs_dict["context_input_ids"],
    context_attention_mask=docs_dict["context_attention_mask"],
    doc_scores=doc_scores,
    decoder_input_ids=labels,
)

AssertionError: 

In [36]:
print(" \u001b[39;49mfaiss_index\u001b[39m.\u001b[39;49msearch(queries, k)\n\u001b[1;32m    376\u001b[0m \u001b[39mreturn\u001b[39;00m BatchedSearchResults(scores, indices\u001b[39m.\u001b[39mastype(\u001b[39mint\u001b[39m))\n\nFile \u001b[0;32m/opt/conda/lib/python3.8/site-packages/faiss/__init__.py:308\u001b[0m, in \u001b[0;36mhandle_Index.<locals>.replacement_search\u001b[0;34m(self, x, k, D, I)\u001b[0m\n\u001b[1;32m    283\u001b[0m \u001b[39m\"\"\"Find the k nearest neighbors of the set of vectors x in the index.\u001b[39;00m\n\u001b[1;32m    284\u001b[0m \n\u001b[1;32m    285\u001b[0m \u001b[39mParameters\u001b[39;00m\n\u001b[0;32m   (...)\u001b[0m\n\u001b[1;32m    304\u001b[0m \u001b[39m    When not enough results are found, the label is set to -1\u001b[39;00m\n\u001b[1;32m    305\u001b[0m \u001b[39m\"\"\"\u001b[39;00m\n\u001b[1;32m    307\u001b[0m n, d \u001b[39m=\u001b[39m x\u001b[39m.\u001b[39mshape\n\u001b[0;32m--> 308\u001b[0m \u001b[39massert\u001b[39;00m d \u001b[39m==\u001b[39m \u001b[39mself\u001b[39m\u001b[39m.\u001b[39md\n\u001b[1;32m    310\u001b[0m \u001b[39massert\u001b[39;00m k \u001b[39m>\u001b[39m \u001b[39m0\u001b[39m\n\u001b[1;32m    312\u001b[0m \u001b[39mif\u001b[39;00m D \u001b[39mis\u001b[39;00m \u001b[39mNone\u001b[39;00m:\n\n\u001b[0;31mAssertionError\u001b[0m: "
)

 faiss_index.search(queries, k)
    376 return BatchedSearchResults(scores, indices.astype(int))

File /opt/conda/lib/python3.8/site-packages/faiss/__init__.py:308, in handle_Index.<locals>.replacement_search(self, x, k, D, I)
    283 """Find the k nearest neighbors of the set of vectors x in the index.
    284 
    285 Parameters
   (...)
    304     When not enough results are found, the label is set to -1
    305 """
    307 n, d = x.shape
--> 308 assert d == self.d
    310 assert k > 0
    312 if D is None:

AssertionError: 


In [15]:
model = model.cuda()

In [25]:
inputs = tokenizer('Who was the Labour candidate in Norwich South?', return_tensors="pt")
input_ids = inputs["input_ids"]
question_hidden_states = model.question_encoder(input_ids.to(device))[0]

In [26]:
# 2. Retrieve
docs_dict = retriever(input_ids.detach().cpu().numpy(), question_hidden_states.detach().cpu().numpy(), return_tensors="pt")
doc_scores = torch.bmm(
    question_hidden_states.detach().cpu().unsqueeze(1), docs_dict["retrieved_doc_embeds"].float().transpose(1, 2)
).squeeze(1)
# 3. Forward to generator


AssertionError: 

In [136]:
docs_dict["doc_ids"]

tensor([[ 45918,  24516,  19557, 173082,  95305]], device='cuda:0')

In [130]:
doc_scores

tensor([[59.8145, 57.2421, 61.1528, 59.3711, 58.7865]])

In [138]:
ds[45918]["text"]

'Warner Bros open nationwide search for Peter Pan actor\nOne rising star could get their lucky break later this month at Wembley Arena, when a nationwide casting call will be taking place to find the new star of Pan.\nThe film, which is directed by Atonement director Joe Wright, will be a live-action remake of JM Barrie\'s classic novel Peter Pan.\nWarner Bros Pictures has extended an invitation to "boys of all ethnicities, aged 11 to 13" to audition for the main role of Peter Pan, the boy who would never grow up.\nThe role is described as "a courageous and adventurous orphaned boy living in a world of fantasy and dreams, which sometimes leads him into trouble."\nThe studio has specified that the aspiring actor must be no taller than 5ft 4inches and must be accompanied by an adult in order to audition.\nThere will be fierce competition for a space in the line on February 23, as queing begins at 9am and closes at 11am, and there is no allowance for overnight queuing or camping.\n'

In [131]:
docs_dict = docs_dict.to(device)

In [132]:
doc_scores = doc_scores.to(device)

In [133]:
generated = model.generate(
    context_input_ids=docs_dict["context_input_ids"],
    context_attention_mask=docs_dict["context_attention_mask"],
    doc_scores=doc_scores,
)
generated_string = tokenizer.batch_decode(generated, skip_special_tokens=True)

/workspace/dller_NLG/.conda/lib/python3.8/site-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [134]:
generated_string

[' 980']

In [127]:
ds[9399]["text"]

'Hercules crash inquest to resume\nLow-level flying\nThe Wiltshire coroner, David Masters, has laid out the main areas for investigation .\nHe will ask whether or not the plane from RAF Lyneham should have been fitted with anti-explosive fuel tanks, what the RAF tactics are for low-level flying during daylight and the availability of intelligence about the threat to aircraft from ground attack.\nExplosive-suppressant foam (ESF) which prevents fuel tanks exploding if they are hit, has been fitted to American Hercules aircraft since the 1960s and Australian Hercules also have it.\nSarah Chapman, sister of Sgt Bob O\'Connor, one of those killed, wrote to Defence Secretary Des Browne in May 2006 to ask why the craft had not been fitted with ESF.\n"The men and women of the British armed forces are not expendable assets who have forfeited their right to life because they chose to take the Queen\'s shilling," she said.\nLegal aid\nIt is therefore not acceptable to deny them any duty of care f

In [143]:
from glob import glob
from datasets import load_dataset
files = glob("../data/raw/*.jsonl")
print(files)
ds_l = [load_dataset("json",data_files=i) for i in files]
ds_sm = ds_l[1]
ds_sm["test"] = ds_l[0]["train"]
ds_sm["validation"] = ds_l[2]["train"]

['../data/raw/streaminqa_valid.jsonl', '../data/raw/streaminqa_train.jsonl', '../data/raw/streaminqa_eval.jsonl']


Using custom data configuration default-d244b28f1b15863e
Found cached dataset json (/root/.cache/huggingface/datasets/json/default-d244b28f1b15863e/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)
100%|██████████| 1/1 [00:00<00:00, 301.06it/s]
Using custom data configuration default-f0b49b41b445f076
Found cached dataset json (/root/.cache/huggingface/datasets/json/default-f0b49b41b445f076/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)
100%|██████████| 1/1 [00:00<00:00, 175.79it/s]
Using custom data configuration default-886a77e275fbf5b2
Found cached dataset json (/root/.cache/huggingface/datasets/json/default-886a77e275fbf5b2/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)
100%|██████████| 1/1 [00:00<00:00, 413.84it/s]


In [144]:
ds_sm

DatasetDict({
    train: Dataset({
        features: ['qa_id', 'question', 'answers', 'answers_additional', 'question_ts', 'evidence_ts', 'evidence_id', 'recent_or_past', 'written_or_generated', 'toxicity_identity_attack', 'toxicity_insult', 'toxicity_profanity', 'toxicity_severe_toxicity', 'toxicity_sexually_explicit', 'toxicity_threat'],
        num_rows: 99402
    })
    test: Dataset({
        features: ['qa_id', 'question', 'answers', 'answers_additional', 'question_ts', 'evidence_ts', 'evidence_id', 'recent_or_past', 'written_or_generated', 'toxicity_identity_attack', 'toxicity_insult', 'toxicity_profanity', 'toxicity_severe_toxicity', 'toxicity_sexually_explicit', 'toxicity_threat'],
        num_rows: 9939
    })
    validation: Dataset({
        features: ['qa_id', 'question', 'answers', 'answers_additional', 'question_ts', 'evidence_ts', 'evidence_id', 'recent_or_past', 'written_or_generated', 'toxicity_identity_attack', 'toxicity_insult', 'toxicity_profanity', 'toxicity_sever

In [161]:
ds_sm = ds_sm.map(lambda examples: {"hash_cleaned": examples["evidence_id"].split("\x00\x01")[1]} , num_proc=2, batched=False)

#1: 100%|██████████| 18189/18189 [00:01<00:00, 12775.29ex/s]


In [162]:
ds_sm

DatasetDict({
    train: Dataset({
        features: ['qa_id', 'question', 'answers', 'answers_additional', 'question_ts', 'evidence_ts', 'evidence_id', 'recent_or_past', 'written_or_generated', 'toxicity_identity_attack', 'toxicity_insult', 'toxicity_profanity', 'toxicity_severe_toxicity', 'toxicity_sexually_explicit', 'toxicity_threat', 'hash_cleaned'],
        num_rows: 99402
    })
    test: Dataset({
        features: ['qa_id', 'question', 'answers', 'answers_additional', 'question_ts', 'evidence_ts', 'evidence_id', 'recent_or_past', 'written_or_generated', 'toxicity_identity_attack', 'toxicity_insult', 'toxicity_profanity', 'toxicity_severe_toxicity', 'toxicity_sexually_explicit', 'toxicity_threat', 'hash_cleaned'],
        num_rows: 9939
    })
    validation: Dataset({
        features: ['qa_id', 'question', 'answers', 'answers_additional', 'question_ts', 'evidence_ts', 'evidence_id', 'recent_or_past', 'written_or_generated', 'toxicity_identity_attack', 'toxicity_insult', 'toxi

In [145]:
ds_sm["train"][1000]["question"]

'Who was the Labour candidate in Norwich South?'

In [165]:
ds_sm["train"][1000]["hash_cleaned"]

'b4715d9b0e42a6b3217076f40252c5ab701eb85ba3e72f6abb58e4ab3d5564a4'

In [160]:
set([type(i) for i in  ds_sm["train"]["evidence_id"]])

{str}

In [156]:
"b4715d9b0e42a6b3217076f40252c5ab701eb85ba3e72f6abb58e4ab3d5564a4" in ds["hash"] 

True

In [171]:
[i for i in ds[ds["hash"].index(ds_sm["train"][1000]["hash_cleaned"])]["text"].split("\n")  if "labour" in i.lower()]

['If the aim is not just to deny Boris Johnson a majority, but to get rid of the Conservatives as the party of government altogether, the best scenario would be Labour polling high, having created a solid anti-Brexit, anti-hard right, anti-austerity tripod.',
 'The fundamental block is the Labour party - obsessed with vote share, it has always opposed alliances of any kind.',
 'In 2017, if it had stood aside in the Isle of Wight - bear in mind that this seat has never returned a Labour MP - the Greens would have repaid them in 12 other constituencies.',
 "While Labour frontbenchers talk constantly to Liberal Democrats - indeed, last week's parliamentary victories couldn't have been won without a fair amount of talking to Tories - nobody at shadow cabinet level is talking about pacts.",
 'The Lib Dems have erected their own barriers, mainly in the form of their performative hostility towards Labour.',
 "Rather, it's designed to underscore their superiority over the Labour party, in the 

In [ ]:
ds[]